In [6]:
using Pkg
Pkg.activate("../scripts/calval")

using IceFloeTracker
using Plots
using Images
using Interpolations

greaterthan05(x) = x .> 0.5 # used for the image resize step and for binarizing images
imrotate_bin_nocrop(x, r) = greaterthan05(collect(imrotate(x, deg2rad(r), method=BSpline(Constant()))))

# Load the images from file
test_images_loc = "../data/test_images_tracker_setup/"
       
# Add labels and get region properties
# labeled_image = label_components(image);
# props = regionprops_table(labeled_image);
props = []
labeled_images = []
for fname in ["001-baffin_bay-20220911-aqua-labeled_floes.png",
              "021_baffin_bay_20120422_aqua_labeled_floes_v1.png",
              "022-barents_kara_seas-20060909-aqua-labeled_floes.png"]
    image = load(joinpath(test_images_loc, fname))
        
    # Add labels and get region properties
    labeled_image = label_components(image)
    push!(labeled_images, labeled_image)
    props_table = regionprops_table(labeled_image)
    IceFloeTracker.addfloemasks!(props_table, greaterthan05.(labeled_image))
    push!(props, props_table)        
    # append!(props, IceFloeTracker.addfloemasks!(props_table, greaterthan0.(labeled_image)))
end
all_props = reduce(vcat, props)
# gotta be a better way to do this -- can't tell why 
# addψs!(props) wouldn't work but this next line does
IceFloeTracker.addψs!([all_props]);

  Activating project at `~/Documents/research/manuscripts/cal-val_ice_floe_tracker/calval_tgrs/scripts/calval`


In [37]:
floe_id = 2
init_floe = all_props[floe_id, :mask]
for rotation in [-27.894736842105267]
    rotated_floe = imrotate_bin_nocrop(init_floe, rotation)
    rotated_props = regionprops_table(label_components(rotated_floe))
    IceFloeTracker.addfloemasks!(rotated_props, rotated_floe) # needed for psi s
    # add filter to drop the length-1 floe.
    
    rotated_props = rotated_props[rotated_props.area .> 1, :]
    IceFloeTracker.addψs!([rotated_props])
    pcorr = IceFloeTracker.corr(all_props[floe_id, "psi"], rotated_props[1, "psi"])                

end


In [36]:
rotated_props = regionprops_table(label_components(rotated_floe))
IceFloeTracker.corr(all_props[floe_id, "psi"], rotated_props[2, "psi"])  

LoadError: ArgumentError: column name :psi not found in the data frame

In [29]:
rotated_props[rotated_props.area .> 1, :]

Row,area,min_row,min_col,max_row,max_col,row_centroid,col_centroid,convex_area,major_axis_length,minor_axis_length,orientation,perimeter
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,4072.0,25,5,87,99,52,52,4249.0,89.5561,59.1012,-1.51827,268.421
